In [2]:
    """
    This code generates a fesibility area for deriving intertidal bathymetry. 
    It computes the area between the the Lowest Astronomical Tide (LAT) and 
    the Mean Seal Level + 10 m, based on a LAT map, and a Bathymetry and 
    elevation GEBCO map. 

        Author: Mario.FuentesMonjaraz@deltares.nl
    """

'\nThis code generates a fesibility area for deriving intertidal bathymetry. \nIt computes the area between the the Lowest Astronomical Tide (LAT) and \nthe Mean Seal Level + 10 m, based on a LAT map, and a Bathymetry and \nelevation GEBCO map. \n\n    Author: Mario.FuentesMonjaraz@deltares.nl\n'

### Define packages

In [3]:
import os
import dask
import xarray as xr
import pyproj
import rioxarray
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import re
import glob


### Define functions

In [14]:
# Define a function to extract the coordinate part of the filename
def extract_coordinates(filename):
    match = re.search(r'gebco_2023_(n-?\d+\.\d+_s-?\d+\.\d+_w-?\d+\.\d+_e-?\d+\.\d+)', filename)
    if match:
        return match.group(1)
    else:
        return None

def assign_projection(ds, epsg=None):

    if not epsg == None:
        proj = pyproj.CRS.from_epsg(int(epsg))
    else:
        proj = pyproj.CRS.from_epsg(int(ds.crs.values.tolist()))

    print(proj,"projection was assigned to the dataset attributes")
    ds.attrs['crs'] = proj
    return ds

def print_ds_properties(rds,epsg=None):
    # Print the grid size
    print("Grid size:", rds.rio.resolution())

    #Print null data
    print("no data:", rds.rio.nodata)

    # Print the projection information
    if rds.rio.crs == None:
        print("There is no projection")
        proj = pyproj.CRS.from_epsg(epsg)
        rds.attrs['crs'] = proj
        rds.rio.set_crs(proj, inplace=True)
    else:
        print("There is projection available")
    
    print("Projection EPSG code is:", rds.rio.crs, "\n")
    return

def change_resolution(ds, new_resolution):
    # Reproject the rioxarray object to the new resolution
    reprojected_ds = ds.rio.reproject(ds.rio.crs, resolution=new_resolution, resampling="bilinear")
    return reprojected_ds

def match_resolution(rds, rds_source):
    # Reproject the rioxarray object to the new resolution
    reprojected_ds = rds.rio.reproject(rds_source.rio.crs, resolution=rds_source.rio.resolution(), resampling= rioxarray.enums.Resampling.bilinear)
    return reprojected_ds

def redefine_null_for_nan(ds, new_null_value):
    # Replace NaN values with the new null value
    ds.values[np.isnan(ds.values)] = new_null_value
    # ds.rio.update({'nodata': new_null_value})
    return ds

def create_gdf_from_geojson_files(input_aoi_data):
    geojson_files = []

    for filename in os.listdir(input_aoi_data):
        if filename.endswith(".geojson"):
            gdf = gpd.read_file(os.path.join(input_aoi_data,filename))
            aoi_id = filename.split('_')
            aoi_id = aoi_id[-1].split('.')[0]
            gdf.insert(1, "aoi", aoi_id)
            geojson_files.append((gdf))

    aoi_gdf = gpd.GeoDataFrame(pd.concat(geojson_files, ignore_index=True)).drop(columns=["id"])
    return aoi_gdf

def clip_raster(rds, geometry):
    rds_clipped = rds.rio.clip(geometry)
    return rds_clipped

def clip_raster_with_gdf(rds, gdf):
    rds_clipped_list = []
    for index, row in gdf.iterrows():
        aoi = row["aoi"]
        try:
            geometry = gdf.iloc[index:index+1].geometry
            rds_clipped = rds.rio.clip(geometry)
            rds_clipped_list.append(rds_clipped)
            print(f"Successful processing row {index} {aoi}")
        except Exception as e:
            print(f"Error processing row {index} {aoi}: {e}")
            rds_clipped_list.append("NaN")
            continue
    print("\n")
    return rds_clipped_list 

def apply_lat_mask_1(depth, lat):

    binary_mask = lat > depth
    depth_lat = depth.where(~binary_mask, other=np.nan)
    return depth_lat 

def apply_lat_mask_2(depth, lat):

    binary_mask = lat <= depth
    depth_lat = depth.where(~binary_mask, other=np.nan)
    return depth_lat 

def apply_hat_mask_1(depth_lat, hat_rds):

    binary_mask =  depth_lat > hat_rds 
    depth_lat_hat = depth_lat.where(~binary_mask, other=np.nan)
    return depth_lat_hat

def apply_hat_mask_2(depth_lat, hat_rds):

    binary_mask =  depth_lat <= hat_rds 
    depth_lat_hat = depth_lat.where(~binary_mask, other=np.nan)
    return depth_lat_hat

def get_coastline(lw_rds, cathegory):
    mask = cathegory
    masked_data = lw_rds.where(mask, other=np.nan)
    return masked_data

def save_mask(depth_lat_hat, name):
    # Assign null values to the created mask
    depth_lat_hat_clean = xr.where((depth_lat_hat.isnull()) | (depth_lat_hat == 9999.0), np.nan, depth_lat_hat)
    depth_lat_hat_ones  = xr.where(depth_lat_hat_clean.isnull(), np.nan, depth_lat_hat_clean / depth_lat_hat_clean)

    # depth_lat.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat.tif"), crs=f"EPSG:{4326}")
    depth_lat_hat.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_{name}.tif"), crs=f"EPSG:{4326}")
    depth_lat_hat_ones.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_{name}_ones.tif"), crs=f"EPSG:{4326}")


### Define packages paths

In [5]:
repository_path = os.path.dirname(os.getcwd())
input_data_path = r"D:\Proyectos2024\Copernicus\Repository\Copernicus\Data" #This folder has to exist with alll the LAT and HAT data that can be downloaded here https://nx1512.your-storageshare.de/s/Xzc6BgHCZ37KbZz
input_aoi_data = r"p:\11209821-cmems-global-sdb\00_miscellaneous\AOIs"
output_data_path = os.path.join(repository_path,"Outputv3")

if not os.path.exists(output_data_path):
    print("Output data path does not exist. Creating directory...")
    os.makedirs(output_data_path)
    print("Output data path created:", output_data_path)
else:
    print("Input data path already exists:", output_data_path)

Input data path already exists: d:\Proyectos2024\Copernicus\Repository\Copernicus\Repository\eo-bathymetry\notebooks\Outputv3


In [6]:
# Get HAT and LAT files
file_paths = glob.glob(os.path.join(input_data_path, 'gebco_2023*.nc'))
filenames = [os.path.basename(document) for document in file_paths]

# Get area of coverage of each gebco file
coordinates_list = [extract_coordinates(filename) for filename in filenames]
coordinates_list = set(coordinates_list)
coordinates_list = list(coordinates_list)

# Reduce the datasets for testing
coordinates_list = ['n90.0_s0.0_w90.0_e180.0']

In [ ]:
# Get HAT and LAT files
file_paths = glob.glob(os.path.join(input_data_path, 'gebco_2023*.nc'))
filenames = [os.path.basename(document) for document in file_paths]

# Get area of coverage of each gebco file
coordinates_list = [extract_coordinates(filename) for filename in filenames]
coordinates_list = set(coordinates_list)
coordinates_list = list(coordinates_list)

# Reduce the datasets for testing
coordinates_list = ['n90.0_s0.0_w90.0_e180.0']

In [54]:

for coordinates in coordinates_list:

    # Read gebco, lat, and hat files
    depthmsl_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'y': 100, 'x':100})

    lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})

    hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})

    lw_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'LandWater15ARC_rusia.tif'))

    print(f'gebco_2023_{coordinates}_depthmsl.nc')
    print_ds_properties(depthmsl_rds)
    print(f'gebco_2023_{coordinates}_lat.nc')
    print_ds_properties(lat_rds)
    print(f'gebco_2023_{coordinates}_hat.nc')
    print_ds_properties(hat_rds)

    # # Remove values below below lat and over hat 
    # depth_lat_1  = apply_lat_mask_1(depthmsl_rds, lat_rds)
    # depth_lat_2 = apply_lat_mask_2(depthmsl_rds, lat_rds)
    # depth_hat_1 = apply_hat_mask_1(depthmsl_rds, hat_rds)
    # depth_hat_2 = apply_hat_mask_2(depthmsl_rds, hat_rds)

    # save_mask(depth_lat_1, 'lat_1')
    # save_mask(depth_lat_2, 'lat_2')
    # save_mask(depth_hat_1, 'hat_1')
    # save_mask(depth_hat_2, 'hat_2')

lat_1 = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_1_ones.tif"))
lat_2 = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_1_ones.tif"))
hat_1 = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_1_ones.tif"))
hat_2 = rioxarray.open_rasterio(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_hat_2_ones.tif"))


gebco_2023_n90.0_s0.0_w90.0_e180.0_depthmsl.nc
Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

gebco_2023_n90.0_s0.0_w90.0_e180.0_lat.nc
Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

gebco_2023_n90.0_s0.0_w90.0_e180.0_hat.nc
Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 



In [70]:
coastline_mask = lw_rds.where(lw_rds == 2.0, other=0)
hat_2_mask     = hat_2.where(hat_2 == 1.0, other=0)
hat_2_mask.rio.to_raster(os.path.join(output_data_path,f"hat_2_mask_v100.tif"), crs=f"EPSG:{4326}")
coastline_mask.rio.to_raster(os.path.join(output_data_path,f"coastline_mask_v100.tif"), crs=f"EPSG:{4326}")

In [88]:
hat_2_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"hat_2_mask_v100.tif"))
coastline_mask = rioxarray.open_rasterio(os.path.join(output_data_path,f"coastline_mask_v100.tif"))

hat_2_mask.rio.write_crs('EPSG:4326', inplace=True)
coastline_mask.rio.write_crs('EPSG:4326', inplace=True)

hat_2_mask, coastline_mask = xr.align(hat_2_mask, coastline_mask)

addition = hat_2_mask + coastline_mask

addition_nan = addition.where(addition != 0, np.nan)
addition_nan.rio.write_crs('EPSG:4326', inplace=True)

addition_nan.rio.to_raster(os.path.join(output_data_path,f"addition_v5000.tif"), crs=f"EPSG:{4326}")

In [79]:
import xarray as xr
import numpy as np
from scipy.ndimage import binary_dilation

# Convert DataArrays to NumPy arrays
layer1_np = lat_2.data
layer2_np = addition_nan.data

# Define the structuring element (8-connectivity)
structure = np.ones((3, 3), dtype=bool)

# Dilate the second layer
dilated_layer2 = binary_dilation(layer2_np, structure=structure)

# Find the touching pixels in the first layer
touching_pixels = (layer1_np & dilated_layer2)

# Convert the result back to a DataArray
touching_pixels_da = xr.DataArray(touching_pixels, dims=layer1_np.dims, coords=layer1_np.coords)

print(touching_pixels_da)

IndexError: tuple index out of range

In [11]:

for coordinates in coordinates_list:

    # Read gebco, lat, and hat files
    depthmsl_ds = xr.open_dataset(os.path.join(input_data_path, f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'lat': 100, 'lon':100})
    depthmsl_rds = rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_depthmsl.nc'), chunks={'y': 100, 'x':100})

    lat_ds = xr.open_dataset(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'lat': 100, 'lon':100})
    lat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_lat.nc'), chunks={'y': 100, 'x':100})

    hat_ds = xr.open_dataset(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'lat': 100, 'lon':100})
    hat_rds = rioxarray.open_rasterio(os.path.join(input_data_path,f'gebco_2023_{coordinates}_hat.nc'), chunks={'y': 100, 'x':100})

    # Set null values in the datasets
    # depthmsl_rds = depthmsl_rds.rio.set_nodata(np.nan)
    # lat_rds      = lat_rds.rio.set_nodata(np.nan)
    # hat_rds      = hat_rds.rio.set_nodata(np.nan)

    print(f'gebco_2023_{coordinates}_depthmsl.nc')
    print_ds_properties(depthmsl_rds)
    print(f'gebco_2023_{coordinates}_lat.nc')
    print_ds_properties(lat_rds)
    print(f'gebco_2023_{coordinates}_hat.nc')
    print_ds_properties(hat_rds)

    # Remove values below below lat and over hat 
    # depth_lat     = apply_lat_mask(depthmsl_rds, lat_rds)
    # depth_lat_hat = apply_hat_mask(depth_lat, hat_rds)

    # depth_lat     = apply_lat_mask(depthmsl_rds, lat_rds)
    depth_lat_hat = apply_hat_mask(depthmsl_rds, hat_rds)

    # Assign null values to the created mask
    depth_lat_hat_clean = xr.where((depth_lat_hat.isnull()) | (depth_lat_hat == 9999.0), np.nan, depth_lat_hat)
    depth_lat_hat_ones  = xr.where(depth_lat_hat_clean.isnull(), np.nan, depth_lat_hat_clean / depth_lat_hat_clean)

    # depth_lat.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat.tif"), crs=f"EPSG:{4326}")
    depth_lat_hat.rio.to_raster(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat.tif"), crs=f"EPSG:{4326}")
    depth_lat_hat_ones.rio.to_raster(os.path.join(output_data_path, f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif"), crs=f"EPSG:{4326}")

    # # Open raster file using rasterio
    # with rasterio.open(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat_ones.tif")) as src:
    #     # Read raster data into numpy array
    #     raster_array = src.read(1)  # Assuming it's a single band raster, adjust if necessary
    #     # Extract transformation metadata
    #     transform = src.transform
    #     # Polygonize raster data
    #     polygons = list(shapes(raster_array, mask=None, transform=transform))
    #     # Convert polygons to Shapely geometries and record pixel values
    #     geometries_with_values = [(shape(polygon), value) for polygon, value in polygons]

    # # Extract geometries and values into separate lists
    # geometries = [geometry for geometry, value in geometries_with_values]
    # values = [value for geometry, value in geometries_with_values]

    # # Convert Shapely geometries and pixel values to GeoDataFrame
    # geo_df = gpd.GeoDataFrame(geometry=geometries, data={'pixel_value': values})

    # geo_df = geo_df[~np.isnan(geo_df['pixel_value'])]
    # geo_df.reset_index(drop=True, inplace=True)

    # # Define the EPSG code for the desired projection
    # epsg_code = 4326  # For example, EPSG code for WGS 84

    # # Assign the projection to the GeoDataFrame
    # geo_df.crs = f"EPSG:{epsg_code}"

    # # Save GeoDataFrame to file
    # # geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat.shp"))
    # geo_df.to_file(os.path.join(output_data_path,f"gebco_2023_{coordinates}_depth_lat_hat.geojson"), driver="GeoJSON", crs=f"EPSG:{epsg_code}")


gebco_2023_n90.0_s0.0_w90.0_e180.0_depthmsl.nc
Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

gebco_2023_n90.0_s0.0_w90.0_e180.0_lat.nc
Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

gebco_2023_n90.0_s0.0_w90.0_e180.0_hat.nc
Grid size: (0.004166666666666668, -0.004166666666666667)
no data: 9999.0
There is projection available
Projection EPSG code is: EPSG:4326 

